Ensemble Learning : Prediction(ex. Classify,Regression) model로부터 얻은 결과들을 종합해 best model을 만드는 것


# Voting Classifiers
- 여러 개의 Trained model에서 가장 자주 등장하는 class들 만으로 집계함.
- hard voting(다수결 방식 분류기) 에 속함.
- Law of large numbers : P(A)+P(B)=1, P(A)>P(B)이면, large number만큼 반복 수행했을 때 N(A)>N(B)일 확률이 매우 높다는 법칙

In [33]:
from sklearn.datasets import make_moons # csv
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

voting_clf = VotingClassifier( # Voting Classifier model로 훈련
    estimators=[
        ('lr', LogisticRegression(random_state=42)),
        ('rf', RandomForestClassifier(random_state=42)),
        ('svc', SVC(random_state=42))
    ]
)
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(random_state=42))])

In [ ]:
# Estimator을 종합적으로 확인하는 방법!
for name, clf in voting_clf.named_estimators_.items():
    print(name, "=", clf.score(X_test, y_test))

lr = 0.864
rf = 0.896
svc = 0.896


In [ ]:
voting_clf.predict(X_test[:]) # Model.predict

array([1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0])

In [ ]:
voting_clf.score(X_test, y_test) # lr, rf, svc보다 점수가 높다.

0.912

# Bagging and Pasting - poewrful Sampling way
- Bagging : 중복을 허용해 반복 추출함. 다양성이 상대적으로 높아지지만 CoVar이 줄어듬.
- Pasting : 중복 허용 없이 반복 추출함.

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500,
                            max_samples=100, n_jobs=-1, random_state=42)
bag_clf.fit(X_train, y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=100,
                  n_estimators=500, n_jobs=-1, random_state=42)

OOB(Out of Bagging) : not-selected data in bagging. Validation set으로 사용된다.

In [ ]:
bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500,
                            oob_score=True, n_jobs=-1, random_state=42)
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_ # obb score

0.896

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.92

Random Patch : train_data와 feature 모두를 Sampling하는 기법
Random Subspace : Sampling을 train_data가 아니라, feature을 기준으로 한다. (모든 train_data 이용)

# Random Forest

Summary) Random Forest는 Bagging/Pasting 을 통해 Ensemble된, Decision Tree model이다.

- Random Forest는 Tree를 만들 때, 최적의 임계값들을 찾아가도록 설계되어 있다.
- splitter='random' : 진짜(extremely) random 하도록 설정 가능한 parameter

- 어떤 Feature이 상대적으로 중요한지 알 수 있다. 이는 평균적으로 Inpurity를 얼마나 감소시키는지를 통해 측정된다.
- Sklearn은 이 값을 'feature_importances_' parameter에 저장한다.

7-6. MNIST Dataset에서 pixel's importancy

<img src='https://drive.google.com/uc?id=18jKomYdpaMkju7ywwD30XsasXuQBdR1f' width='350' height='250'>
<left>

# Boosting

AdaBoost(p.282) : 이전 model에서 underfitting했던 train_sample의 weight를 높이는 방법.

<img src = 'https://drive.google.com/uc?id=1WNaamGN0a3qUTiqeGzxFN5N7JNPzUsk1'>
<left>
- Gradient Descent는 parameter를, AdaBoost는 weight를 수정한다는 점에서 차이가 있지만, 점차 접근한다는 점에서 공통점이 있다.

Gradient Boosting(p.286) : AdaBoost와 달리, Residual error(잔여 오차)를수정하면서 새로운 model을 학습시킨다. (AdaBoost는 weight 수정)

In [ ]:
# DecisionTreeRegressor 실습
import numpy as np
from sklearn.tree import DecisionTreeRegressor

np.random.seed(42)
X = np.random.rand(100, 1) - 0.5
y = 3 * X[:, 0] ** 2 + 0.05 * np.random.randn(100)  # y = 3x² + Gaussian noise

tree_reg1 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg1.fit(X, y)

DecisionTreeRegressor(max_depth=2, random_state=42)

In [ ]:
y2 = y - tree_reg1.predict(X) # Residual error 발생
tree_reg2 = DecisionTreeRegressor(max_depth=2, random_state=43)
tree_reg2.fit(X, y2)

DecisionTreeRegressor(max_depth=2, random_state=43)

In [ ]:
y3 = y2 - tree_reg2.predict(X) # Residual error 발생
tree_reg3 = DecisionTreeRegressor(max_depth=2, random_state=44)
tree_reg3.fit(X, y3)

DecisionTreeRegressor(max_depth=2, random_state=44)

In [ ]:
# 3개 Tree를 합함으로써 Ensemble함.
X_new = np.array([[-0.4], [0.], [0.5]])
sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

array([0.49484029, 0.04021166, 0.75026781])

<left>
<img src = 'https://drive.google.com/uc?id=1m3JW6nRxglypHbhEBJzwAn1VVeOi3ZQh' >
<left>
- left, right : Tree model, Ensembled Tree model
- parameter, 'learning_rate'는 각 Tree의 기여도를 조정함. 낮은 값은 더 많은 Tree를 요구하지만, 일반적으로 성능은 좋아진다.

- Stochastic Gradient Boosting : 각 Tree가 훈련될 때 train_sample_rate를 지정하는 parameter 추가 가능함.
- HGB(Histogram-based Gradient Boosting) : Sample size가 수만 개 이상일 때 매우 효율적임. Sample을 Histogram(막대 도표)을 통해 분할하고 최적의 boundary를 찾는다.

# Stacking
- Ensemble은 '특정 통계량'을 바탕으로 취합했음을 상기하자.
- Stacking은 Blender(=Meta Learner='통계량'을 종합한 model) layer을 추가하는 개념이다.
- 성능은 조금 향상되지만, cost와 복잡성이 증가한다.

In [ ]:
from sklearn.ensemble import StackingClassifier

stacking_clf = StackingClassifier(
    estimators=[ # lr, rf, svc를 보여주는 layer이 추가됨
        ('lr', LogisticRegression(random_state=42)),
        ('rf', RandomForestClassifier(random_state=42)),
        ('svc', SVC(probability=True, random_state=42))
    ],
    final_estimator=RandomForestClassifier(random_state=43),
    cv=5  # number of cross-validation folds
)
stacking_clf.fit(X_train, y_train)

StackingClassifier(cv=5,
                   estimators=[('lr', LogisticRegression(random_state=42)),
                               ('rf', RandomForestClassifier(random_state=42)),
                               ('svc', SVC(probability=True, random_state=42))],
                   final_estimator=RandomForestClassifier(random_state=43))

In [ ]:
stacking_clf.score(X_test, y_test)

0.928

# Practice 8. Mnist Estimation : non-ensembling vs ensembling

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from keras.datasets import mnist
from sklearn.metrics import accuracy_score # Model test

mnist_data = mnist.load_data()
# train(+val):test = 6:1 로 분할
(x_train, y_train), (x_test, y_test) = mnist_data

In [2]:
# import models
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import svm

# Rename models
rfc = RandomForestClassifier()
etc = ExtraTreesClassifier()
svm = svm.SVC()

In [9]:
x_train = x_train.reshape((-1, x_train.shape[1]*x_train.shape[2]))
x_test = x_test.reshape((-1, x_test.shape[1]*x_test.shape[2]))

Fit to the models

In [12]:
rfc_model = rfc.fit(x_train, y_train)

In [13]:
etc_model = etc.fit(x_train, y_train)

Test the models

In [32]:
rfc.score(x_test, y_test), etc.score(x_test, y_test)

(0.9688, 0.973)

Voting System을 통해 Model esmble

In [35]:
voting_esm = VotingClassifier(
    estimators=[
        ('rf', RandomForestClassifier(random_state=42)),
        ('etc', ExtraTreesClassifier(random_state=42))
    ]
)

voting_esm.fit(x_train, y_train)

ValueError: Found input variables with inconsistent numbers of samples: [60000, 375]

In [38]:
np.shape(x_train)

(60000, 784)